In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e

In [ ]:
from google.colab import userdata

In [ ]:
import wandb


In [ ]:
wandb.login(key=userdata.get('WANDB_API_KEY'))
hf_token = userdata.get('HF_TOKEN')
!huggingface-cli login --token {hf_token}

In [ ]:
data_dir = "/gdrive/MyDrive/finetune"

base_model_id = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"


In [ ]:
pip install json_repair

In [ ]:
!pip -q install -U "numpy>=2.0.0" \
                     "transformers==4.48.3" \
                     "accelerate>=0.34.2" \
                     "tokenizers>=0.20.3" \
                     "huggingface_hub>=0.25.2" \
                     "safetensors>=0.4.5" einops
# !pip -q install bitsandbytes==0.43.3


In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
!pip install -U numpy
!pip install -U transformers accelerate


In [ ]:
device = "cpu"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
story = """
كشفت منصة “wired” المتخصصة بالمواضيع التقنية والذكاء الاصطناعي عن انتشار ظاهرة سرقة المحتوى الإخباري عبر الإنترنت، إذ رصدت موقعًا ينشر مقالات منسوخة، ومدعومة بالذكاء الاصطناعي، تظهر في مراكز متقدمة بنتائج البحث على محرك “جوجل”.

المنصة قالت إن القصة بدأت عندما بحث أحد محرريها عن قصص إخبارية تتعلق بتعامل شركة “أدوبي” مع محتوى المستخدمين، وتوظيفه في تدريب نماذجها للذكاء الاصطناعي، عندما اكتشف أن مقالًا منشورًا في الموقع الإخباري يظهر في المركز الثاني داخل تبويب الأخبار لدى “جوجل”، بينما يظهر مقال من مدونة مغمورة تحمل اسم “Syrus” في المرتبة الأولى، رغم أن المقال يظهر وكأنه نسخة مسروقة من المقال الأصلي مع تغييرات تحريرية طفيفة.

وأضافت أنه عند النقر على الرابط التشعبي لنتيجة البحث، ظهر موقع إلكتروني يعج بالرسائل العشوائية والمقالات المسروقة التي أعيد تجميعها، وكثير منها باستخدام الرسوم التوضيحية المنشأة بالذكاء الاصطناعي


في مقال البريد العشوائي هذا، نُسخ مقال للمنصة نفسها كاملًا، مع تغييرات طفيفة في الصياغة، حتى إن الاقتباسات في المقال الأصلي حُذفت من المقال المنسوخ، وفرض المقال نفسه رابطًا تشعبيًا وحيدًا في أسفل صفحة “الويب”، يؤدي إلى النسخة الأصلية من المقال، كمصدر وحيد للمعلومات الواردة فيه.

وفق التقرير، تواصلت “wired” مع مدير المدونة، وتلقت تأكيدًا بأن وكالة تسويق إيطالية أنشأتها، واستخدمت أداة ذكاء اصطناعي كجزء من عملية الكتابة.

وقال مدير المدونة الذي يستخدم اسم “Daniele Syrus” لـ”wired”، إن عملية إنشاء المحتوى تتضمن أدوات ذكاء اصطناعي، تتركز مهمتها على تحليل وتجميع المعلومات من مصادر متعددة “مع احترام الملكية الفكرية دائمًا”، من خلال وضع رابط المقال الأصلي أسفل المقال على الموقع.

وتجنبت شركة “جوجل” التعليق على تقرير “wired”، وفق المنصة، بينما أوضحت ميجان فارنسورث، المتحدثة باسم “جوجل”، أن سياسات التعامل مع المحتوى المخالف لدى محرك البحث تحظر إنشاء محتوى منخفض القيمة، وغير أصلي، بغرض رفع ترتيب مواقع “الويب” في نتائج البحث لدى “بحث جوجل” (Google Search)، مؤكدة أن الشركة تتخذ إجراءات ضد المواقع التي لا تلتزم بسياساتها، وفق المنصة.

بدورها، مديرة تحسين محركات البحث في وكالة التسويق “Amsive”، ليلي راي، ذكرت أن مشكلة المحتوى المزعج المنتشر على “جوجل” صعبة التفسير، مشيرة إلى أن بعض العملاء اشتكوا من إعادة صياغة مقالاتهم بالذكاء الاصطناعي بشكل مشابه لمحتواهم الأصلي، ولكن مع تغييرات طفيفة.

وبالنظر إلى سياسات “جوجل”، انتهكت المدونة قواعد النشر فيما يتعلق باستخراج البيانات عبر الإنترنت، وفق “wired”، إذ تعتبر “جوجل” نسخ المحتوى من مواقع أخرى وتعديله قليلًا فقط خرقًا للقواعد.



."""

In [ ]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified","AI"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")

class NewsDetails(BaseModel):
    story_title: str = Field(..., min_length=5, max_length=300,
                             description="A fully informative and SEO optimized title of the story.")

    story_keywords: List[str] = Field(..., min_items=1,
                                      description="Relevant keywords associated with the story.")

    story_summary: List[str] = Field(
                                    ..., min_items=1, max_items=5,
                                    description="Summarized key points about the story (1-5 points)."
                                )

    story_category: StoryCategory = Field(..., description="Category of the news story.")

    story_entities: List[Entity] = Field(..., min_items=1, max_items=10,
                                        description="List of identified entities in the story.")


In [ ]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                NewsDetails.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Story Details:",
            "```json"
        ])
    }
]

In [ ]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., min_length=5, max_length=300,
                                  description="Suggested translated title of the news story.")
    translated_content: str = Field(..., min_length=5,
                                    description="Translated content of the news story.")

targeted_lang = "English"

translation_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content":  "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
            "",

            "## Targeted Language:",
            targeted_lang,
            "",

            "## Translated Story:",
            "```json"

        ])
    }
]

In [ ]:
from huggingface_hub import snapshot_download
target_dir = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"

_ = snapshot_download(
    repo_id="Qwen/Qwen2.5-0.5B-Instruct",
    local_dir=target_dir,
    local_dir_use_symlinks=False,
    token=None,
)

base_model_id = target_dir

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch_dtype,
    trust_remote_code=True,
)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

base_model_id = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto" if torch.cuda.is_available() else None,
    torch_dtype=torch_dtype,
    trust_remote_code=True,
)


In [ ]:
model

In [ ]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
response

In [ ]:
text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

In [ ]:
from huggingface_hub import snapshot_download

target_dir = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-7B-Instruct"

_ = snapshot_download(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    local_dir=target_dir,
    local_dir_use_symlinks=False,
    token=None,

print("✅ Model downloaded to:", target_dir)

In [ ]:
pip install -U bitsandbytes

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id_t = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id_t,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)

print("✅ Model loaded successfully")

In [ ]:
device = model.device  # cuda:0


# translation_messages = [
#   {"role": "system", "content": "You are a helpful assistant that replies in Arabic."},
#   {"role": "user", "content": "ترجم للجملة التالية للإنجليزية: الذكاء الاصطناعي رائع."}
# ]

text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=None,
        top_p=None,
        top_k=None,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

gen_ids = outputs[0, inputs["input_ids"].shape[1]:]
response = tokenizer.decode(gen_ids, skip_special_tokens=True)

print(response)


In [ ]:
device = "cpu"
torch_dtype = torch.float32

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map=device,
    torch_dtype=torch_dtype,
    trust_remote_code=True
)

print("✅ Model loaded successfully")


In [ ]:
!pip install -U bitsandbytes
!pip install --upgrade numpy


In [ ]:
import json
import random
from os.path import join

data_dir = "/gdrive/MyDrive/finetune"

raw_data_path = join(data_dir, "finetune","datasets", "news-sample.jsonl")

raw_data = []
with open(raw_data_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        raw_data.append(json.loads(line.strip()))

random.Random(101).shuffle(raw_data)

print(f"✅ Raw data loaded: {len(raw_data)} samples")


In [ ]:
raw_data[0]['content']

In [ ]:
import json, json_repair, re, os
from os.path import join
from tqdm.auto import tqdm
import torch

data_dir = "/gdrive/MyDrive/finetune"
save_to = join(data_dir, "finetune","datasets", "sft.jsonl")
os.makedirs(os.path.dirname(save_to), exist_ok=True)

prompt_tokens = 0
completion_tokens = 0

def extract_json(text: str):
    m = re.search(r"```json\s*(\{.*\}|\[.*\])\s*```", text, flags=re.S)
    if not m:
        m = re.search(r"(\{.*\}|\[.*\])", text, flags=re.S)
    if not m:
        return None
    raw = m.group(1)
    try:
        return json.loads(raw)
    except Exception:
        try:
            return json_repair.loads(raw)
        except Exception:
            return None

def ask_teacher(messages, max_new_tokens=512, temperature=0.2):
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    device = model.device
    inputs = tokenizer(text, return_tensors="pt").to(device)
    input_len = inputs["input_ids"].shape[1]

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature is not None and temperature > 0),
            temperature=temperature if temperature else None,
            top_p=0.9 if temperature else None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = out[0, input_len:]
    reply = tokenizer.decode(gen_ids, skip_special_tokens=True)

    in_tok = inputs["input_ids"].numel()
    out_tok = gen_ids.numel()
    return reply, in_tok, out_tok

ix = 0
for story in tqdm(raw_data):
    sample_details_extraction_messages = [
        {
            "role": "system",
            "content": "\n".join([
                "You are an NLP data paraser.",
                "You will be provided by an Arabic text associated with a Pydantic scheme.",
                "Generate the output in the same story language.",
                "You have to extract JSON details from text according the Pydantic details.",
                "Extract details as mentioned in text.",
                "Do not generate any introduction or conclusion."
            ])
        },
        {
            "role": "user",
            "content": "\n".join([
                "## Story:",
                story['content'].strip(),
                "",
                "## Pydantic Details:",
                json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
                "",
                "## Story Details:",
                "```json"
            ])
        }
    ]

    try:
        llm_response, in_tok, out_tok = ask_teacher(
            sample_details_extraction_messages,
            max_new_tokens=512,
            temperature=0.2
        )
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            torch.cuda.empty_cache()
            llm_response, in_tok, out_tok = ask_teacher(
                sample_details_extraction_messages,
                max_new_tokens=256,
                temperature=0.2
            )
        else:
            continue

    llm_resp_dict = extract_json(llm_response)
    if not llm_resp_dict:
        continue

    with open(save_to, "a", encoding="utf-8") as f:
        f.write(json.dumps({
            "id": ix,
            "story": story['content'].strip(),
            "task": "Extract the story details into a JSON.",
            "output_scheme": json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
            "response": llm_resp_dict
        }, ensure_ascii=False, default=str) + "\n")

    ix += 1
    prompt_tokens += in_tok
    completion_tokens += out_tok

    if (ix % 3) == 0:
        print(f"Iteration {ix}: prompt_tokens={prompt_tokens}, completion_tokens={completion_tokens}")

print(f"✅ Done. Written samples: {ix}")
print(f"📄 Saved to: {save_to}")


In [ ]:
import json
import random
from os.path import join

sft_data_path = join(data_dir, "finetune","datasets", "sft.jsonl")
llm_finetunning_data = []

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Story:",
            rec["story"],

            "# Task:",
            rec["task"],
            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)

print(f"Total data samples: {len(llm_finetunning_data)}")


In [ ]:
train_sample_sz = 2700

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "finetune","datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "finetune","datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(data_dir,"finetune", "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/youtube-resources/llm-finetuning/models/checkpoint-1500
output_dir: /gdrive/MyDrive/finetune/finetune/llm-finetuning/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "abdelati88/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint


In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -r requirements.txt


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

In [ ]:
!ls /content/LLaMA-Factory/


In [ ]:
!ls /content/LLaMA-Factory/examples/train_lora/


In [ ]:
%cd /content/LLaMA-Factory
!pip install -r requirements.txt


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

In [ ]:
!llama_factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!python3 /content/LLaMA-Factory/llama_factory_cli.py train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!pip install -r /content/LLaMA-Factory/requirements.txt


In [ ]:
!ls /content/LLaMA-Factory/


In [ ]:
!find . -name "llama_factory_cli.py"


In [ ]:
!pip install transformers datasets wandb huggingface_hub


In [ ]:
!python3 /content/LLaMA-Factory/llama_factory_cli.py train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

In [ ]:
!mkdir -p /content/LLaMA-Factory/examples/train_lora

news_finetune_yaml = """
### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: /content/drive/MyDrive/finetune/models/
logging_steps: 10
save_steps: 500
plot_loss: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "abdelati88/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint
"""

with open("/content/LLaMA-Factory/examples/train_lora/news_finetune.yaml", "w") as f:
    f.write(news_finetune_yaml)


In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
%cd /content/LLaMA-Factory


In [ ]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


In [ ]:
%cd /content/LLaMA-Factory
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!pip install -r /content/LLaMA-Factory/requirements.txt



In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
transformers-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
%cd /content/LLaMA-Factory
huggingface-cli upload --repo_id <your-username>/<model-name> --model_path ./content/LLaMA-Factory


In [ ]:
pip install transformers

In [ ]:
!transformers-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!huggingface-cli login


In [ ]:
!python /content/LLaMA-Factory/src/train.py --config /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
finetuned_model_id = "/gdrive/MyDrive/finetune/finetune/llm-finetuning/models"
model.load_adapter(finetuned_model_id)

In [ ]:
def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_resp(translation_messages)